In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
import os
import re
import itertools

In [ ]:
os.chdir('')

In [ ]:
for file in os.listdir("./"):
  if not os.path.isfile(f"./{file}"):
    print(os.listdir(f"./{file}")[0])

In [ ]:
#Matrix
matrix = np.zeros(100).reshape(25,4)
dataframe = pd.DataFrame(matrix, columns = ["Petiole Length", "Leaf Margin", "Leaf Shape", "Number of  Leafs"])
index = []
row = 0
#Preprocessing
for file in os.listdir(os.getcwd()):
  if not os.path.isfile(os.path.join(os.getcwd(), file)):
    os.chdir(os.path.join(os.getcwd(), file))
    for f in os.listdir(os.getcwd()):
        x = re.search("xlsx$", f)
        if  x != None:
            data = pd.read_excel(f)
            accesion = data["Accesion"][0]
            print(accesion)
            index.append(accesion)
            data.drop(["Replicate","Accesion"], axis = 1, inplace = True)
            #Discretizing
            discretizer = KBinsDiscretizer(n_bins=3, encode ="ordinal", strategy='uniform')
            data["Number of  Leafs"] = discretizer.fit_transform(data["Number of  Leafs"].values.reshape(-1,1)).astype(int)
            data.replace(0, "Low", inplace = True)
            data.replace(1, "Medium", inplace = True)
            data.replace(2, "High", inplace = True)
            #Fill Dataframe
            dataframe.iloc[row,:] = data.mode(axis = 0).values.tolist()[0]
            row += 1
    os.chdir("../")

In [ ]:
#reindex
dataframe_values = dataframe.values
dataframe_columns = dataframe.columns.tolist() 
dataframe = pd.DataFrame(dataframe_values, index = index ,columns=dataframe_columns)
dataframe

In [ ]:
pd.DataFrame.to_csv(dataframe, "traits_matrix.csv")

In [ ]:
#Similarity function matrix

def sim_matriz(data):
  """Creates similarity matrix"""
  matrix = np.zeros(data.shape[0]**2).reshape(data.shape[0], data.shape[0])
  for f in np.arange(data.shape[0]):
    for f2 in np.arange(data.shape[0]):
      in_common = 0
      for col in np.arange(data.shape[1]):
        if data.iloc[f, col] == data.iloc[f2, col]:
          in_common += 1
        else:
          continue 
      matrix[f, f2] = in_common / data.shape[1]
  return matrix

In [ ]:
similarity_frame = pd.DataFrame(sim_matriz(dataframe), index = dataframe.index.tolist(), columns = dataframe.index.tolist())
similarity_frame

In [ ]:
pd.DataFrame.to_csv(similarity_frame, "similarity_matrix.csv")

In [ ]:
n_row = 0
sim_5 = pd.DataFrame(np.zeros(106260).reshape((53130),2), columns = ["Combination", "Dissimilarity"])
for comb in itertools.combinations(similarity_frame.columns.tolist(), 5):
    similarity = 0
    for comb_ in itertools.combinations(comb, 2):
        sim = similarity_frame.loc[comb_[0], comb_[1]]
        similarity += sim
    row = [comb, (1-(similarity / 10))]
    sim_5.iloc[n_row, :] = row
    n_row += 1
sim_5 = sim_5.sort_values(by=['Dissimilarity'], ascending=False)
pd.DataFrame.to_csv(sim_5, "dissimilarity_5.csv")